# Preprocesamiento de valores perdidos

En este *notebook* vamos a aplicar distintos tipos de preprocesamiento de valores perdidos.

In [8]:
import pandas as pd

## Lectura con valores perdidos

Lo primero es una lectura correcta de los valores. Si no tenemos en cuenta que existen valores perdidos, y cómo están representados en el CSV, interpretará dicha columna como valor de cadena.

In [9]:
df = pd.read_csv("ejemplo_missing.csv")

In [10]:
df.head()

,BI-RADS,Age,Shape,Margin,Density,Severity
0,5,67,L,5,3,maligno
1,4,43,R,1,?,maligno
2,5,58,I,5,3,maligno
3,4,28,R,1,3,benigno
4,5,74,R,5,?,maligno


In [11]:
df.dtypes

BI-RADS     object
Age         object
Shape       object
Margin      object
Density     object
Severity    object
dtype: object

La solución a esto es indicar que el "?" indica un valor perdido, y eso se hace con:

In [12]:
df = pd.read_csv("ejemplo_missing.csv", na_values=["?"])

In [13]:
df.head()

,BI-RADS,Age,Shape,Margin,Density,Severity
0,5.0,67.0,L,5.0,3.0,maligno
1,4.0,43.0,R,1.0,NaN,maligno
2,5.0,58.0,I,5.0,3.0,maligno
3,4.0,28.0,R,1.0,3.0,benigno
4,5.0,74.0,R,5.0,NaN,maligno


In [14]:
df.dtypes

BI-RADS     float64
Age         float64
Shape        object
Margin      float64
Density     float64
Severity     object
dtype: object

# Opción simple, borrar

Lo más simple es eliminar cualquier columna cuyo atributo esté perdido o no sea un valor válido. Para los perdidos se puede usar dropna().

In [15]:
df2 = df.copy()
df2.dropna().shape

(847, 6)

In [16]:
df.shape

(961, 6)

Es sencillo, pero tiene el problema de perder muchos datos.

# Opción alternativa: Reemplazarlos 

La otra opción es reemplazar el valor perdido con un valor, puede ser:
    
- Valor más frecuente en dicha columna.
- Valor promedio.
- Valor que no perjudique la media y la std.

Para ello podemos usar sklearn.impute, vamos a hacer un ejemplo.

In [17]:
from sklearn import impute

In [18]:
df.Density.head()

0    3.0
1    NaN
2    3.0
3    3.0
4    NaN
Name: Density, dtype: float64

In [19]:
df.Density.tail(20)

941    3.0
942    3.0
943    1.0
944    3.0
945    3.0
946    3.0
947    3.0
948    3.0
949    3.0
950    3.0
951    3.0
952    3.0
953    3.0
954    3.0
955    3.0
956    3.0
957    3.0
958    3.0
959    3.0
960    3.0
Name: Density, dtype: float64

Primero creamos un SimpleImputer, y lo aplicamos. Como el SimpleImputer trabaja con una matriz de vectores usamos [], y luego usaremos la primera fila devuelta.

In [20]:
help(impute.SimpleImputer)

Help on class SimpleImputer in module sklearn.impute._base:

class SimpleImputer(_BaseImputer)
 |  SimpleImputer(*, missing_values=nan, strategy='mean', fill_value=None, verbose=0, copy=True, add_indicator=False)
 |  
 |  Imputation transformer for completing missing values.
 |  
 |  Read more in the :ref:`User Guide <impute>`.
 |  
 |  .. versionadded:: 0.20
 |     `SimpleImputer` replaces the previous `sklearn.preprocessing.Imputer`
 |     estimator which is now removed.
 |  
 |  Parameters
 |  ----------
 |  missing_values : int, float, str, np.nan or None, default=np.nan
 |      The placeholder for the missing values. All occurrences of
 |      `missing_values` will be imputed. For pandas' dataframes with
 |      nullable integer dtypes with missing values, `missing_values`
 |      should be set to `np.nan`, since `pd.NA` will be converted to `np.nan`.
 |  
 |  strategy : string, default='mean'
 |      The imputation strategy.
 |  
 |      - If "mean", then replace missing values u

In [21]:
imputer = impute.SimpleImputer(strategy="most_frequent")

values = imputer.fit_transform([df.Density.values])
values

array([[3., 3., 3., 3., 3., 3., 3., 1., 3., 3., 3., 2., 2., 3., 3., 3.,
        3., 1., 1., 3., 3., 3., 3., 3., 3., 2., 3., 3., 2., 3., 3., 2.,
        2., 1., 3., 2., 3., 4., 3., 2., 3., 3., 1., 3., 3., 3., 3., 3.,
        3., 3., 2., 3., 3., 3., 3., 2., 3., 2., 1., 3., 2., 3., 1., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 2., 3., 2., 3., 3.,
        3., 2., 3., 3., 2., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 2., 3., 3., 3., 3., 3., 4., 4., 3., 2., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 2., 3.,
        3., 3., 4., 3., 2., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 2., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 1., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 1., 3., 3., 3.,
        3., 3., 3., 3., 3., 1., 3., 3., 2., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 2., 3., 3., 3., 2., 3., 3., 1., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 2., 3., 3., 3., 3., 

Ahora actualizamos el atributo, usando los nuevos valores.

In [22]:
df.Density.update(pd.Series(values[0]))

Comprobamos que ya no hay NaN en el atributo.

In [23]:
df.Density.head()

0    3.0
1    3.0
2    3.0
3    3.0
4    3.0
Name: Density, dtype: float64

In [24]:
df.Density.tail(20)

941    3.0
942    3.0
943    1.0
944    3.0
945    3.0
946    3.0
947    3.0
948    3.0
949    3.0
950    3.0
951    3.0
952    3.0
953    3.0
954    3.0
955    3.0
956    3.0
957    3.0
958    3.0
959    3.0
960    3.0
Name: Density, dtype: float64

Comprobamos que no haya Nan en el df original.

In [25]:
df.head()

,BI-RADS,Age,Shape,Margin,Density,Severity
0,5.0,67.0,L,5.0,3.0,maligno
1,4.0,43.0,R,1.0,3.0,maligno
2,5.0,58.0,I,5.0,3.0,maligno
3,4.0,28.0,R,1.0,3.0,benigno
4,5.0,74.0,R,5.0,3.0,maligno


In [26]:
df3 = df.copy()
df3.dropna()

,BI-RADS,Age,Shape,Margin,Density,Severity
0,5.0,67.0,L,5.0,3.0,maligno
1,4.0,43.0,R,1.0,3.0,maligno
2,5.0,58.0,I,5.0,3.0,maligno
3,4.0,28.0,R,1.0,3.0,benigno
4,5.0,74.0,R,5.0,3.0,maligno
...,...,...,...,...,...,...
956,4.0,47.0,O,1.0,3.0,benigno
957,4.0,56.0,I,5.0,3.0,maligno
958,4.0,64.0,I,5.0,3.0,benigno
959,5.0,66.0,I,5.0,3.0,maligno


In [27]:
df2 = df2.dropna()

In [28]:
df2.shape

(847, 6)

In [29]:
df2.Density.describe()

count    847.000000
mean       2.909091
std        0.370292
min        1.000000
25%        3.000000
50%        3.000000
75%        3.000000
max        4.000000
Name: Density, dtype: float64

In [30]:
df3 = df3.dropna()
df3.shape

(907, 6)

In [31]:
df3.Density.describe()

count    907.000000
mean       2.912900
std        0.373101
min        1.000000
25%        3.000000
50%        3.000000
75%        3.000000
max        4.000000
Name: Density, dtype: float64